# calculates and outputs in html the subset related stats required for the WDC subset page
e.g. http://webdatacommons.org/structureddata/2020-12/stats/schema_org_subsets.html

In [ ]:
import re
import gzip
from os import listdir
from os.path import isfile, join
from multiprocessing import Pool, Process, Manager
from tqdm import tqdm
import tldextract
import os

input_path = "/work-ceph/aprimpel/WDC_Extraction_2020/7_c_schema_json_full_no_enc_issues/combined_files/"

def getstatsofsubset(subsetfile):
    
    quadcounter = 0
    distinct_urls = set()
    distinct_domains = set()
    schema_dict = dict()
    size = os.path.getsize(input_path+subsetfile)/(1024*1024)
    with gzip.open(input_path+subsetfile,'rt') as f:
        for line in f:
            quadcounter +=1
            #line = line_.decode('utf8')
            
            if ("<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>" in line):

                schema_type = line.split()[-3][1:-1]
                if(schema_type not in schema_dict):
                    schema_dict[schema_type]=0
                
                schema_dict[schema_type] = schema_dict[schema_type]+1
                url = line.split()[-2][1:-1]
                distinct_urls.add(url)
                domain_extract  = tldextract.extract(url)
                domain = domain_extract.domain+'.'+domain_extract.suffix
                distinct_domains.add(domain)
    return(quadcounter, len(distinct_urls), len(distinct_domains), schema_dict, size, subsetfile)       

In [ ]:
files_ = [f for f in listdir(input_path) if isfile(join(input_path, f))]

html_stats = ""
pool = Pool(20)
for result in tqdm(pool.imap(func=getstatsofsubset, iterable=files_), total=len(files_)):
    filename = result[5]
    schema_subset = filename.replace("schema_","").replace(".gz","")
    top_related_classes= "</td><td>"

    for k in sorted(result[3], key=result[3].get, reverse=True)[:5]: 
        top_related_classes= top_related_classes+k+" ("+str(f"{result[3][k]:,}")+")"+"</br>"

    #print(filename)
    #print(top_related_classes)
    #print("---------")
    html_stats_file = ("<tr><th><a href=\"http://schema.org/"+schema_subset+"\">"+schema_subset+
          "</a></th><td> Quads: "+str(f"{result[0]:,}")+"</br> URLs: "+str(f"{result[1]:,}")+
          "</br> Hosts: "+str(f"{result[2]:,}")+"</br>"+top_related_classes+"</td><td>"+str(round(result[4], 2))+" MB </td><td><a href=\"http://data.dws.informatik.uni-mannheim.de/structureddata/2020-12/quads/classspecific/md/"+filename+"\">"+filename+"</a> (<a href=\"http://data.dws.informatik.uni-mannheim.de/structureddata/2020-12/quads/classspecific/md/"+filename.replace(".gz",".txt")+"\">sample</a>)</td></tr>\n")
    
    print(html_stats_file)
    html_stats+=html_stats_file
      
